# Business Understanding

Dengan semakin banyaknya konten yang tersedia di platform streaming film, pengguna dihadapkan pada pilihan yang sangat beragam. Tanpa bantuan sistem yang tepat, pengguna mungkin kesulitan menemukan konten yang sesuai dengan preferensi mereka, yang dapat berdampak pada pengalaman pengguna (user experience) dan keterlibatan mereka dengan platform. Dari sudut pandang bisnis, menampilkan konten yang relevan secara konsisten dapat meningkatkan waktu tayang pengguna (watch time), retensi pelanggan, dan pada akhirnya meningkatkan pendapatan melalui langganan yang lebih lama atau konsumsi konten berbayar yang lebih banyak.

**Problem Statements**
- Bagaimana solusi bagi pengguna agar tidak kesulitan dalam menemukan konten film sesuai dengan preferensi mereka pada platform streaming?  
- Bagaimana memberikan suatu pertimbangan bagi pengguna untuk bertahan dalam menggunakan platform streaming?

**Goals**
- Mengembangkan sistem rekomendasi film berbasis content-based filtering.
- Memberikan output top-N rekomendasi film yang mirip dengan film yang dipilih pengguna.

**Solution statements**
- Menggunakan pendekatan Content-Based Filtering, hal ini dilakukan dengan memanfaatkan deskripsi film, genre, aktor, dan sutradara sebagai fitur konten. Kemiripan antar film akan diukur berdasarkan kesamaan fitur-fitur ini.
- Menggunakan teknik pemrosesan bahasa alami (NLP) seperti tokenization, lowercasing, dan stemming pada deskripsi film.
- Menggunakan algoritma Term Frequency-Inverse Document Frequency (TF-IDF) untuk mengubah teks deskripsi menjadi vektor numerik.
- Menghitung cosine similarity antara vektor representasi film untuk mengukur kemiripan konten.

# Data Understanding

Dataset yang digunakan merupakan dataset yang berisikan informasi detail mengenai berbagai film, termasuk judul, tanggal rilis, genre, sinopsis, aktor, sutradara, rating, dan lain-lain. Dataset ini diperoleh dari Kaggle.
- Dataset ini terdiri dari dua file yaitu `movies.csv` dan `credits.csv`
- Dataset memiliki 4800+ baris
- Link Dataset: [TMDB 5000 Movie Dataset](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata)

Variabel-variabel pada TMDB 500 Movie Dataset adalah sebagai berikut. Variabel yang dijelaskan merupakan variabel yang dipilih sebab dianggap penting dalam pembangunan model sistem rekomendasi.
- id : kode unik dari masing-masing film
- title : judul film
- genres : pengelompokan film berdasarkan kesamaan gaya atau tema
- keywords : kata kunci yang menunjukkan identikal dari film
- overview : deskripsi gambaran umum dari film
- cast : aktor/aktris yang berperan dalam film
- crew : sekelompok orang yang berada di balik layar pembuatan film

## Import Library

In [1]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Data Loading

In [2]:
df_movies = pd.read_csv("/content/tmdb_5000_movies.csv")
df_credits = pd.read_csv("/content/tmdb_5000_credits.csv")

## Exploratory Data Analysis

Dalam Exploratory Data Analysis (EDA) ini akan dilakukan dua tahap analisis, yaitu:
- Univariate Analysis (menganalisis masing-masing informasi dari dataset)
- Data Preprocessing (melakukan pengolahan pada bagian isi data)

### Univariate Analysis

1. Movies Dataset

In [3]:
# Menampilkan dataset movies
df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [5]:
print("Banyak judul film:", len(df_movies.title.unique()))

Banyak judul film: 4800


Dari analisis di atas terdapat 20 kolom fitur yang terdiri dari kurang lebih 4803 baris pada dataset movie. Banyaknya judul film pada dataset berjumlah 4800 film, hal ini menandakan adanya indikasi duplikasi data atau missing value, sebab maksimal baris data pada kolom "title" berjumlah 4803 judul

2. Credits Dataset

In [6]:
# Menampilkan dataset credits
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [7]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


Dari analisis di atas terdapat 4 kolom fitur dan 4803 baris pada dataset credits

### Data Preprocessing

1. Mengubah dan Memastikan Nama Kolom

In [8]:
# Mengubah nama kolom "movie_id" pada dataset credits menjadi "id"
df_credits = df_credits.rename(columns={"movie_id": "id"})
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      4803 non-null   int64 
 1   title   4803 non-null   object
 2   cast    4803 non-null   object
 3   crew    4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB


Hal ini bertujuan untuk menyelaraskan nama kolom id pada kedua dataset sebelum melakukan penggabungan (merge) dataset

In [9]:
# Memastikan kolom id dan title di kedua dataset sama
df_movies[['id', 'title']].head()

,id,title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [10]:
df_credits[['id', 'title']].head()

,id,title
0,19995,Avatar
1,285,Pirates of the Caribbean: At World's End
2,206647,Spectre
3,49026,The Dark Knight Rises
4,49529,John Carter


In [11]:
# Menghapus kolom title di dataset credits
df_credits = df_credits.drop(columns=['title'])

Membandingkan kolom "id" dan "title" pada kedua dataset untuk memastikan bahwa nomor id dan judul film sama dengan melihat beberapa sampel baris. Kemudian melakukan drop(hapus) kolom "title" pada salah satu dataset untuk melakukan proses penggabungan agar kolom tidak double.

2. Menggabungkan Dataset

In [12]:
# Menggabungkan dataset movies dan credits
df_merge = df_movies.merge(df_credits, on="id", how="left")
df_merge.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [13]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

Menggabungkan kedua dataset untuk proses persiapan data, salah satunya seleksi fitur. Sebab kita membutuhkan beberapa fitur yang ada pada dataset credits, sehingga perlu dilakukan penggabungan (merge). Menghasilkan 22 kolom fitur pada dataset baru

# Data Preparation

1. Seleksi Fitur, Penanganan Missing Value, Pengecekan Duplikasi

In [14]:
# Seleksi fitur yang akan digunakan
df = df_merge[["id", "title", "genres", "keywords", "overview", "cast", "crew"]]
df.isnull().sum()

,0
id,0
title,0
genres,0
keywords,0
overview,3
cast,0
crew,0


In [15]:
df.dropna(inplace=True)
df.shape

<ipython-input-15-1e8442348ac0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


(4800, 7)

In [16]:
print("Jumlah duplikasi data: ", df.duplicated().sum())
df.head(1)

Jumlah duplikasi data:  0


,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


Membuat dataset baru, dengan memilih fitur yang dibutuhkan saja. Kemudian melakukan pengecekan terhadap missing value dan duplikasi pada fitur. Terindikasi 3 missing value pada overview. Sebab gambaran umum dari film tidak kita ketahui, lebih baik baris missing value di hapus. Sehingga dataset terdiri dari 4800 baris dan 7 kolom penting

2. Mengekstrak dan Menggabungkan List

In [17]:
# Ekstrasi fitur
def extract_list(text):
    try:
        data = json.loads(text)
        return [item['name'] for item in data]
    except:
        return []

df['genres'] = df['genres'].apply(extract_list)
df['keywords'] = df['keywords'].apply(extract_list)
df['cast'] = df['cast'].apply(lambda x: extract_list(x)[:3]) # Ambil 3 aktor pertama
df['crew'] = df['crew'].apply(lambda x: [item['name'] for item in json.loads(x) if item['job'] == 'Director'][0] if any(item['job'] == 'Director' for item in json.loads(x)) else '')

df.head(1)

<ipython-input-17-700c7ef73993>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(extract_list)
<ipython-input-17-700c7ef73993>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].apply(extract_list)
<ipython-input-17-700c7ef73993>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

,id,title,genres,keywords,overview,cast,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron


 Beberapa kolom seperti genres, keywords, cast, dan crew berisi data dalam format JSON. Perlu dilakukan ekstraksi untuk mendapatkan daftar genre, kata kunci, nama aktor, dan nama sutradara.

In [18]:
# Penggabungan Fitur Teks
def combine_features(row):
    overview = row['overview'].split() if isinstance(row['overview'], str) else []
    return ' '.join(overview + row['genres'] + row['keywords']*2 + row['cast']*3 + [row['crew']]*2)

df["tags"] = df.apply(combine_features, axis=1)

<ipython-input-18-f7c6344e751a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tags"] = df.apply(combine_features, axis=1)


In [19]:
df.head(1)

,id,title,genres,keywords,overview,cast,crew,tags
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"In the 22nd century, a paraplegic Marine is di..."


Menggabungkan teks dari kolom overview, daftar genre, keywords, nama aktor, dan nama sutradara menjadi satu representasi teks (kolom "tags") untuk setiap film. Ini akan menjadi input utama untuk perhitungan kemiripan konten. Selain itu memberikan pembobotan pada fitur yang dianggap berpengaruh seperti fitur keywords, cast, dan crew

In [20]:
df_final = df[["id", "title", "tags"]]
df_final.head(1)

,id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."


Membuat dataset final yang terdiri dari id film, judul film, dan representasi film (tags)

3. Normalisasi Teks List (Cleaning Text)

In [21]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower() # Mengubah teks menjadi huruf kecil
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Hilangkan karakter selain huruf dan spasi
    tokens = word_tokenize(text) # Tokenisasi (memecah kalimat jadi kata-kata)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words] # Hilangkan stopwords & lemmatize
    return ' '.join(tokens) # Gabungkan kembali jadi string

df_final['tags'].apply(clean_text)

,tags
0,nd century paraplegic marine dispatched moon p...
1,captain barbossa long believed dead come back ...
2,cryptic message bond past sends trail uncover ...
3,following death district attorney harvey dent ...
4,john carter warweary former military captain w...
...,...
4798,el mariachi want play guitar carry family trad...
4799,newlywed couple honeymoon upended arrival resp...
4800,signed sealed delivered introduces dedicated q...
4801,ambitious new york attorney sam sent shanghai ...


In [22]:
df_final['tags'] = df_final['tags'].apply(clean_text)
df_final.head()

<ipython-input-22-558cfb6ad65c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['tags'] = df_final['tags'].apply(clean_text)


,id,title,tags
0,19995,Avatar,nd century paraplegic marine dispatched moon p...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed dead come back ...
2,206647,Spectre,cryptic message bond past sends trail uncover ...
3,49026,The Dark Knight Rises,following death district attorney harvey dent ...
4,49529,John Carter,john carter warweary former military captain w...


Melakukan pembersihan (normalisasi) teks pada kolom "tags" sebelum masuk ke tahap modeling. Hal ini dilakukan agar model dapat mengolah representasi film untuk memperoleh hasil konten yang serupa
- Lowercase: Mengubah teks menjadi huruf kecil
- Re.sub: Menghilangkan karakter yang bukan spasi dan huruf
- Tokenisasi: Memecah teks menjadi unit-unit kata (token)
- Stopword Removal: Menghapus kata-kata umum dalam bahasa inggris yang tidak memiliki banyak informasi semantik
- Lemmatization: Mengembalikan kata-kata ke dalam bentuk dasarnya, misalnya stories menjadi story

# Model Development

Pada tahap ini, pengembangan model dilakukan dengan menggunakan
- Representasi Fitur (kolom "tags") yaitu gabungan antara fitur overview, genres, keywords, cast, dan crew.
- Menerapkan algoritma TF-IDF terhadap fitur gabungan untuk mengubahnya menjadi vektor numerik.
- Menghitung cosine similarity antara vektor TF-IDF dari fitur yang telah divektorkan terhadap film. Cosine similarity mengukur sudut antara dua vektor, dengan nilai 1 menunjukkan kemiripan sempurna dan nilai 0 menunjukkan tidak ada kemiripan
- Untuk memberikan rekomendasi untuk film tertentu, sistem akan mencari film-film lain dengan nilai cosine similarity tertinggi terhadap film tersebut.

In [23]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words='english', ngram_range=(1,2))
tfidf_matrix = tfidf_vectorizer.fit_transform(df_final['tags'])

In [24]:
# Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [25]:
# Membuat reverse mapping dari indeks ke judul film
indices = pd.Series(df_final.index, index=df_final['title']).drop_duplicates()

In [26]:
# Fungsi untuk Mendapatkan Rekomendasi dengan Skor Cosine Similarity
def get_recommendations_with_scores(title, cosine_sim=cosine_sim, df=df_final, indices=indices):
    try:
        idx = indices[title]
    except KeyError:
        print(f"Film '{title}' tidak ditemukan dalam dataset.")
        return []

    # Dapatkan skor similaritas untuk film tersebut dengan semua film lain
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Urutkan film berdasarkan skor similaritas
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Dapatkan skor untuk 10 film yang paling mirip (tidak termasuk film itu sendiri)
    sim_scores = sim_scores[1:11]

    # Dapatkan indeks film dan skor similaritas
    movie_indices = [i[0] for i in sim_scores]
    similarity_scores = [i[1] for i in sim_scores]

    # Kembalikan DataFrame yang berisi judul dan skor similaritas
    df_recommendations = pd.DataFrame({'title': df_final['title'].iloc[movie_indices], 'similarity_score': similarity_scores})
    return df_recommendations

# Evaluasi

In [27]:
# Contoh penggunaan sistem rekomendasi 1 (satu) dengan skor
movie_title = "Iron Man"
recommendations_with_scores = get_recommendations_with_scores(movie_title)

print(f"\nRekomendasi film untuk '{movie_title}' dengan perhitungan Cosine Similarity:")
print(recommendations_with_scores)


Rekomendasi film untuk 'Iron Man' dengan perhitungan Cosine Similarity:
                                  title  similarity_score
79                           Iron Man 2          0.638752
31                           Iron Man 3          0.586217
7               Avengers: Age of Ultron          0.518114
16                         The Avengers          0.492665
26           Captain America: Civil War          0.482212
182                             Ant-Man          0.362603
126                Thor: The Dark World          0.361242
174                 The Incredible Hulk          0.360789
129                                Thor          0.311287
169  Captain America: The First Avenger          0.285891


In [28]:
# Contoh penggunaan sistem rekomendasi 2 (dua) dengan skor
movie_title = "The Fast and the Furious"
recommendations_with_scores = get_recommendations_with_scores(movie_title)

print(f"\nRekomendasi film untuk '{movie_title}' dengan Cosine Similarity:")
print(recommendations_with_scores)


Rekomendasi film untuk 'The Fast and the Furious' dengan Cosine Similarity:
                 title  similarity_score
44           Furious 7          0.514879
500   2 Fast 2 Furious          0.437285
204          Fast Five          0.428334
1083      Babylon A.D.          0.315712
568                xXx          0.314293
574           S.W.A.T.          0.283731
2775    Find Me Guilty          0.276235
1742    Brick Mansions          0.242254
1347       A Man Apart          0.240491
1395     Resident Evil          0.240084


**Metode Evaluasi**

Evaluasi ini tidak menggunakan metrik numerik, sebab evaluasi lebih difokuskan terhadap relevansi rekomendasi berdasarkan fitur metadata.

Evaluasi ini hanya dapat dilihat dari cosine similarity (skor kemiripan) terhadap film yang dipilih.
- Lebih dari 60: Sangat mirip (biasanya sekuel atau series)
- 0.50 - 0.60: Masih sangat relevan
- 0.30 - 0.50: Mirip secara tema/genre/universe
- < 30: Sudah mulai kurang relevan, bisa secara tema/genre/universe/pemeran/sutradara

**Evaluasi penggunaan sistem rekomendasi 1 (satu)**
- Top 5 rekomendasi sangat baik — semuanya berada dalam narasi atau konflik yang dekat dengan Tony Stark/Iron Man.
- Skor cosine similarity di atas 0.4 bisa dianggap cukup bagus untuk TF-IDF sederhana.
- Film seperti Iron Man 2, Iron Man 3, dan Civil War memang secara narasi dan karakter sangat dekat.
- Urutan 6 ke bawah memang memiliki skor cosine rendah tetapi masih ada relevansi tema dan universe yang sama dari alur film Iron Man

**Evaluasi penggunaan sistem rekomendasi 2 (dua)**
- Top 3 rekomendasi sangat baik - semuanya relevan sebab berasal dari produksi yang sama, memiliki karakter utama yang sama, tema dan gaya penyutradaraan mirip.
- Rekomendasi film urutan 3 ke atas memiliki narasi yang sangat dekat sebab film-film ini merupakan bagian dari cerita sekuensial film yang dipilih
- Urutan 4 ke bawah memiliki skor cosine sedang ke rendah, sebab relevansi nya hanya pemeran utama, genre, dan aksi yang sama. Tidak memiliki relevansi yang kuat dengan cerita Fast & Furious
